In [1]:
import cv2
import torch
import numpy as np
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split

In [10]:
num_out_feature = 128
p = 0.5

class RESNET18(nn.Module):
    def __init__(self):
        super(RESNET18, self).__init__()

        self.resnet = nn.Sequential(*list(resnet18.children())[:-1])
        
        self.fc1 = nn.Linear(resnet18.fc.in_features, num_out_feature)
        self.fc2 = nn.Linear(num_out_feature, 30)

        self.batchnorm = nn.BatchNorm1d(num_out_feature)

        self.dropout = nn.Dropout(p)

    def forward(self, x):
        x = self.resnet(x)
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.batchnorm(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [3]:
def resize_and_pad_image(image, output_size=(256, 256)):
    h, w, _ = image.shape
    pad_width = abs(h-w) // 2
    
    if h > w:
        image = cv2.copyMakeBorder(image, 0, 0, pad_width, pad_width, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    elif h < w:
        image = cv2.copyMakeBorder(image, pad_width, pad_width, 0, 0, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    
    image = cv2.resize(image, output_size)

    return Image.fromarray(image)

In [4]:
transform = transforms.Compose([
    transforms.Lambda(lambda x: resize_and_pad_image(np.array(x), (224, 224))),
    transforms.ToTensor(),
])

In [6]:
all_data_tensor = torch.load('./Dataset Tensors/data_tensor.pth')
all_labels_tensor = torch.load('./Dataset Tensors/labels_tensor.pth')

dataset = []
for i in range(len(all_labels_tensor)):
    dataset.append((all_data_tensor[i], all_labels_tensor[i]))


torch.manual_seed(42)
total_size = len(dataset)
train_size = int(total_size * 0.7)  # 70% for training
val_size = int(total_size * 0.2)    # 20% for validation

test_size = total_size - train_size - val_size  # Remaining 10% for testing

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [7]:
letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'call', 'del', 'space', 'thumbsup']

In [11]:
prediction_count = {element: 0 for element in set(letters)}

model = torch.load("Training Models/dropout_full_resnet18_2.pth")
model.eval()
corr = 0
total = 0

# for data in dataloader:
for data, label in test_loader:
    predictions = model(data)
    probabilities = F.softmax(predictions, dim=1)
    prob, pred = torch.max(F.softmax(predictions, dim=1), 1)

    corr += pred.eq(label.view_as(pred)).sum().item()
    total += data.shape[0]

    # prediction_count[letters[pred.item()]] += 1

In [16]:
print("Accuracy: {:.2f}%".format(corr / total * 100))

Accuracy: 99.88%


In [15]:
model.cpu()

RESNET18(
  (resnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [82]:
sorted_prediction_count = {k: v for k, v in sorted(prediction_count.items(), key=lambda item: item[1], reverse=True)}

sorted_prediction_count

{'A': 2,
 'N': 0,
 'space': 0,
 'E': 0,
 'P': 0,
 'del': 0,
 'D': 0,
 'V': 0,
 'F': 0,
 'Q': 0,
 'B': 0,
 'W': 0,
 'Y': 0,
 'Z': 0,
 'H': 0,
 'L': 0,
 'S': 0,
 'M': 0,
 'G': 0,
 'J': 0,
 'K': 0,
 'O': 0,
 'T': 0,
 'thumbsup': 0,
 'R': 0,
 'C': 0,
 'X': 0,
 'U': 0,
 'I': 0,
 'call': 0}